<a href="https://colab.research.google.com/github/Mohammad-Amirifard/Traffic-forecasting/blob/main/Notebooks/EDA_traffic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'traffic:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5137002%2F8588395%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240602%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240602T223405Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D603ffb5833c53caec6777a4177712aa6ee11688902fd6dee837195740661c9664bc47f52e03dfdddd90e6f06bbf701141ec15ab93a8d6777bb58465a75d4aae604d5b3eaf0aed438c6b748557833914d8d4d63d240ed9ba491a53a19b78279aa93b184bd8763f70c9ffa5418132bc83b1d6675581222b325f764cee8eea5c86fcc24c8474993a0cd15bc77c05793d48beb0524891213c78ac1f0fe4b350a08482d180d9f165036c3d947b8b7dc00d59b3d192ef1a7bbcdd4e766c6d82ef586399f8aafaed75fa89663be6d82f2fc79a2f2b838d60c49ebe6bb280ac29b24d3cdf164f08f4ff93a95887c0f8ae34c8f44760548db33651157503e77d6ec1cbe48,files-names:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5137134%2F8588631%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240602%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240602T223405Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D851dc4b85dd63a5a9d90024168e441600b325b63733b6ccceded3436daa9f48ca535ace154421ddb39790b1afbf10ded0b269616804dff5a4445300c9e03f4ffbad3b445816f67a5268fc73f02b9a1779f1b0740605750d7317dbeba5ab20ca36279f9244bb477064ef14cbf3c5ce4efb1faea07578db30d21be210614f5f4c13e5d7b8ba4d7b04dff3f7e6fb9340ffa783adf79e15655a39f308c6705f2db231d7d9af45d9d99d9547e3f911c3dfae4ee89765e6420608a74b7f7384f6c533ba17d4f9040a60ea8ef5c9cfb28602bf7d8f2953530d2417fce66beaa4d35362a5fd3918cda18dcdd2814701896202f42f7015a00a91e9bd985fc5a5f86c55ec4'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xml.etree.ElementTree as ET

In [ ]:
with open('/kaggle/input/files-names/files_names.txt','r') as file:
    files_names =file.read().splitlines()


In [ ]:
files_names[0]

'IntraTM-2005-01-01-00-30.xml'

In [ ]:
# Parse the XML file

df = pd.DataFrame({
    'src_id': [],
    'dst_id':[],

})


#!ls -1 '/kaggle/input/traffic' | sort -n > files_names.txt

folder = files_names
# Initialize lists to store data

for index,file in enumerate(folder) :
    print(index)
    file = '/kaggle/input/traffic/'+ file
    tree = ET.parse(file)
    root = tree.getroot()
    identifier_name = file.split('.')[-2].split('/')[-1].split('IntraTM-')[-1]



    # Extract data
    for src in root.findall(".//src"):
        src_id = src.get('id')
        for dst in src.findall('dst'):

            dst_id = dst.get('id')
            value = dst.text

            src_ids.append(src_id)
            dst_ids.append(dst_id)

            values.append(value)

    df[identifier_name]=values

df['src_id'] = src_ids
df['dst_id'] = dst_ids

    # Display the DataFrame
display(df)

0
1


ValueError: Length of values (475) does not match length of index (477)

IntraTM-2005-01-01-00-30.xml
IntraTM-2005-01-01-00-45.xml
IntraTM-2005-01-01-01-00.xml
IntraTM-2005-01-01-01-15.xml
IntraTM-2005-01-01-01-30.xml
IntraTM-2005-01-01-01-45.xml
IntraTM-2005-01-01-02-00.xml
IntraTM-2005-01-01-02-15.xml
IntraTM-2005-01-01-02-30.xml
IntraTM-2005-01-01-02-45.xml
IntraTM-2005-01-01-03-00.xml
IntraTM-2005-01-01-03-15.xml
IntraTM-2005-01-01-03-30.xml
IntraTM-2005-01-01-03-45.xml
IntraTM-2005-01-01-04-00.xml
IntraTM-2005-01-01-04-15.xml
IntraTM-2005-01-01-04-30.xml
IntraTM-2005-01-01-04-45.xml
IntraTM-2005-01-01-05-00.xml
IntraTM-2005-01-01-05-15.xml
IntraTM-2005-01-01-05-30.xml
IntraTM-2005-01-01-05-45.xml
IntraTM-2005-01-01-06-00.xml
IntraTM-2005-01-01-06-15.xml
IntraTM-2005-01-01-06-30.xml
IntraTM-2005-01-01-06-45.xml
IntraTM-2005-01-01-07-00.xml
IntraTM-2005-01-01-07-15.xml
IntraTM-2005-01-01-07-30.xml
IntraTM-2005-01-01-07-45.xml
IntraTM-2005-01-01-08-00.xml
IntraTM-2005-01-01-08-15.xml
IntraTM-2005-01-01-08-30.xml
IntraTM-2005-01-01-08-45.xml
IntraTM-2005-0

,Source,Destination,Value
333,14,12,35275.9822
334,14,13,79.3156
335,14,19,30.3378
336,14,23,14.0889
337,14,8,12.4444
338,14,18,3.1556
339,14,4,3536.8444
340,14,1,309.9111
341,14,5,13.7600
342,14,3,202.8089
